In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as T
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import math
import time
from typing import Callable, Optional, Tuple, Dict, Any

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using:", device)

Using: cuda


In [ ]:
def vr_zo_adamm(
    # required
    model_or_predict: Any,            # either a torch.nn.Module OR a callable predict_fn(batch_tensor)->logits
    x0,                               # numpy array or torch tensor with shape (C,H,W) or (1,C,H,W)
    true_label: int,

    # ZO / VR hyperparams
    mu: float = 1e-2,
    q: int = 64,
    Q_ref: int = 256,
    inner_m: int = 20,
    epochs: int = 5,
    dist_weight: float = 0.01,

    # Adam-like
    alpha: float = 0.05,
    beta1: float = 0.9,
    beta2: float = 0.99,
    eps: float = 1e-8,
    v_init: float = 1e-6,

    # projection / constraints
    constrained: bool = True,
    lb: float = -0.5,
    ub: float = 0.5,

    # batching / device
    device: Optional[torch.device] = None,
    forward_batch_size: int = 256,   # split model batches when evaluating many probes
    rng_seed: int = 1234,

    # stopping / budgets
    early_stop: bool = True,
    stop_threshold: float = 0.0,
    max_queries: int = 2000,
    max_distortion: Optional[float] = None,
    patience: int = 40,
    tol: float = 1e-3,
    smooth_window: int = 20,
    flat_threshold: float = 1e-3,

    # misc
    predict_fn: Optional[Callable[[torch.Tensor], torch.Tensor]] = None,
    verbose: bool = True,
) -> Tuple[np.ndarray, Dict[str,Any]]:
    """
    VR–ZO–AdaMM generalized for arbitrary PyTorch model.

    Returns:
      delta (numpy array shape d,) : perturbation vector to add to x0 (flattened)
      diagnostics: dict with keys: queries, history: {loss, queries, dist}, stop_reason, time_elapsed

    Usage:
       - If your model needs special preprocessing (normalization), pass predict_fn(batch_tensor)->logits that does it.
       - x0 can be numpy (C,H,W) or torch (C,H,W) or batched (1,C,H,W).
       - true_label: integer label for untargeted attack.

    Query counting convention: 1 forward on B images -> +B queries.
    """

    # ---------- prepare device and predict function ----------
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # if user passed a model, make default predict_fn
    if predict_fn is None:
        if callable(model_or_predict):
            # assume model_or_predict is a torch.nn.Module
            model = model_or_predict
            model.eval()
            model.to(device)
            def _predict_fn(batch: torch.Tensor) -> torch.Tensor:
                # expects batch on same device
                with torch.no_grad():
                    return model(batch)
            predict = _predict_fn
        else:
            raise ValueError("Either provide a model (nn.Module) as model_or_predict or pass predict_fn explicitly.")
    else:
        predict = predict_fn

    # ---------- prepare x0 and shapes ----------
    if isinstance(x0, torch.Tensor):
        x0_t = x0.detach().cpu().float()
    else:
        x0_t = torch.from_numpy(np.array(x0)).float()

    # support x0 shaped (C,H,W) or (1,C,H,W)
    if x0_t.ndim == 3:
        C,H,W = x0_t.shape
    elif x0_t.ndim == 4 and x0_t.shape[0] == 1:
        _, C,H,W = x0_t.shape
        x0_t = x0_t.squeeze(0)
    else:
        raise ValueError("x0 must have shape (C,H,W) or (1,C,H,W)")

    d = int(C*H*W)
    x_shape = (C,H,W)

    # move base x0 to device once
    x0_dev = x0_t.to(device)

    torch.manual_seed(rng_seed)
    rng = np.random.RandomState(rng_seed)

    # ---------- helpers ----------
    def sample_dirs(q_local:int) -> torch.Tensor:
        U = torch.randn((q_local, d), device=device)
        norms = U.norm(dim=1, keepdim=True)
        norms = torch.where(norms == 0, torch.ones_like(norms), norms)
        return U / norms  # (q, d)

    def batched_predict(batch_imgs: torch.Tensor) -> torch.Tensor:
        """
        Run predict on batch_imgs (N,C,H,W) in sub-batches sized forward_batch_size.
        Returns logits tensor (N, n_classes)
        Also increments queries by N (handled by callers).
        """
        n = batch_imgs.shape[0]
        outs = []
        for i in range(0, n, forward_batch_size):
            chunk = batch_imgs[i:i+forward_batch_size].to(device)
            with torch.no_grad():
                out = predict(chunk)
            outs.append(out.detach().cpu())
        return torch.cat(outs, dim=0).to(device)

    def imgs_from_deltas(deltas: torch.Tensor) -> torch.Tensor:
        # deltas: (q, d) or (B, d) on device -> returns (q, C, H, W)
        return (x0_dev.unsqueeze(0) + deltas.view(-1, C, H, W))

    # ZO symmetric estimator using batched forwards:
    def symmetric_zo_grad(x: torch.Tensor, mu_t: float, directions: torch.Tensor) -> Tuple[torch.Tensor,int]:
        """
        x: tensor (d,) on device
        directions: (q, d) on device
        returns: grad_est (d,) on device, queries_added (int)
        """
        q_local = int(directions.shape[0])

        delta_plus  = x.unsqueeze(0) + mu_t * directions   # (q,d)
        delta_minus = x.unsqueeze(0) - mu_t * directions   # (q,d)

        imgs_plus  = imgs_from_deltas(delta_plus).cpu()   # move to CPU for batched_predict which expects CPU tensors maybe
        imgs_minus = imgs_from_deltas(delta_minus).cpu()

        batch = torch.cat([imgs_plus, imgs_minus], dim=0)  # (2q, C, H, W)
        # run forward in chunks
        logits = batched_predict(batch)  # (2q, nclass)
        # increment queries: each image counted
        q_inc = batch.shape[0]

        probs = F.softmax(logits, dim=1)  # (2q, C)

        # compute margin per sample: log p_true - log max_other
        probs_clone = probs.clone()
        probs_clone[:, true_label] = float('-inf')
        p_true = probs[:, true_label]            # (2q,)
        p_other = probs_clone.max(dim=1).values  # (2q,)

        margin = torch.log(p_true + 1e-12) - torch.log(p_other + 1e-12)  # (2q,)

        # distortion term per delta
        dist_plus  = (delta_plus.norm(dim=1)**2).detach()  # (q,)
        dist_minus = (delta_minus.norm(dim=1)**2).detach() # (q,)

        f_plus  = (margin[:q_local] + dist_weight * dist_plus).detach()
        f_minus = (margin[q_local:] + dist_weight * dist_minus).detach()


        dir_deriv = (f_plus - f_minus) / (2.0 * mu_t)  # (q,)
        grad_est = (d / float(q_local)) * (directions.t() @ dir_deriv)  # (d,)
        grad_est = grad_est.to(device)

        return grad_est, q_inc

    def mahalanobis_projection(x: torch.Tensor, sqrt_vhat: torch.Tensor) -> torch.Tensor:
        # x, sqrt_vhat, x0_dev: all on device
        z = sqrt_vhat * x
        Lb = sqrt_vhat * (lb - x0_dev.view(-1))
        Ub = sqrt_vhat * (ub - x0_dev.view(-1))
        z_clipped = torch.max(torch.min(z, Ub), Lb)
        return z_clipped / sqrt_vhat

    # ---------- initialization ----------
    x_snap = torch.zeros(d, device=device)
    x_curr = torch.zeros(d, device=device)
    m = torch.zeros(d, device=device)
    v = v_init * torch.ones(d, device=device)
    vhat = v.clone()

    query_count = 0
    loss_trace = []
    queries_trace = []
    dist_trace = []
    best_val = float("inf")
    no_improve = 0
    start_time = time.time()

    mu_t = float(mu)

    # ---------- stopping helper ----------
    def check_stopping(val_f: float, delta: torch.Tensor) -> Tuple[bool,str]:
        nonlocal best_val, no_improve, query_count
        if val_f < stop_threshold:
            return True, "margin reached"
        if (max_distortion is not None) and (float(delta.norm()) > max_distortion):
            return True, "distortion limit exceeded"
        if val_f < best_val - tol:
            best_val = val_f
            no_improve = 0
        else:
            no_improve += 1
        if no_improve > patience:
            return True, "patience exhausted"
        # moving average flattening
        loss_trace.append(val_f)
        if len(loss_trace) > 2 * smooth_window:
            recent = np.mean(loss_trace[-smooth_window:])
            previous = np.mean(loss_trace[-2*smooth_window:-smooth_window])
            if previous - recent < flat_threshold:
                return True, "moving-average plateau"
        if query_count >= max_queries:
            return True, "query budget exhausted"
        return False, None

    # ---------- main loop ----------
    stop_reason = None
    hist = {"loss": [], "queries": [], "dist": [], "time": []}

    for epoch in range(epochs):
        # reference gradient at snapshot
        dirs_ref = sample_dirs(Q_ref)
        g_ref, q_inc = symmetric_zo_grad(x_snap, mu_t, dirs_ref)
        query_count += q_inc

        # reset moments (optionally you can keep them)
        m.zero_()
        v = v_init * torch.ones_like(v)
        vhat = v.clone()

        for t in range(inner_m):
            directions = sample_dirs(q)
            g_curr, q_inc = symmetric_zo_grad(x_curr, mu_t, directions); query_count += q_inc
            g_snap, q_inc = symmetric_zo_grad(x_snap, mu_t, directions); query_count += q_inc

            g_vr = g_curr - g_snap + g_ref

            # Adam/AMSGrad
            m = beta1 * m + (1.0 - beta1) * g_vr
            v = beta2 * v + (1.0 - beta2) * (g_vr * g_vr)
            vhat = torch.max(vhat, v)

            step = alpha * m / (torch.sqrt(vhat) + eps)
            x_new = x_curr - step

            if constrained:
                sqrt_vhat = torch.sqrt(vhat)
                x_new = mahalanobis_projection(x_new, sqrt_vhat)

            x_curr = x_new

            # evaluate objective once (single forward)
            img = (x0_dev + x_curr.view(C,H,W)).unsqueeze(0)  # (1, C, H, W)
            logits = batched_predict(img)   # returns (1, nclass) on device
            query_count += 1
            probs = F.softmax(logits, dim=1)[0]
            probs_clone = probs.clone()
            probs_clone[true_label] = float('-inf')
            p_true = probs[true_label].item()
            p_other = probs_clone.max().item()
            margin = np.log(p_true + 1e-12) - np.log(p_other + 1e-12)
            distortion = float(x_curr.norm()**2)
            val = margin + dist_weight * distortion

            elapsed = time.time() - start_time
            hist["loss"].append(val)
            hist["queries"].append(query_count)
            hist["dist"].append(np.sqrt(distortion))
            hist["time"].append(elapsed)

            if verbose and (t % 10 == 0):
                print(f"[epoch {epoch} step {t}] f={val:.6f} queries={query_count} time={elapsed:.1f}s dist={np.sqrt(distortion):.3f}")

            if early_stop:
                stop, reason = check_stopping(val, x_curr)
                if stop:
                    stop_reason = reason
                    if verbose:
                        print("Early stop:", reason, "final f:", val, "queries:", query_count)
                    delta_final = x_curr.detach().cpu().numpy()
                    diagnostics = {
                        "queries": int(query_count),
                        "stop_reason": stop_reason,
                        "history": hist,
                        "time_elapsed": time.time() - start_time
                    }
                    return delta_final, diagnostics

        x_snap = x_curr.clone()

    # finished all epochs
    delta_final = x_curr.detach().cpu().numpy()
    diagnostics = {
        "queries": int(query_count),
        "stop_reason": stop_reason or "completed_epochs",
        "history": hist,
        "time_elapsed": time.time() - start_time
    }
    if verbose:
        print("Finished epochs; stop_reason:", diagnostics["stop_reason"], "queries:", diagnostics["queries"])
    return delta_final, diagnostics


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as T
from torchvision.models import resnet18
from tqdm import tqdm
def resnet18_cifar10():
    model = resnet18(weights=None)

    model.conv1 = nn.Conv2d(
        3, 64, kernel_size=3, stride=1, padding=1, bias=False
    )
    model.bn1 = nn.BatchNorm2d(64)
    model.relu = nn.ReLU(inplace=True)
    model.maxpool = nn.Identity()

    # Replace FC layer
    model.fc = nn.Linear(512, 10)
    return model

model = resnet18_cifar10().to(device)

In [ ]:
transform_train = T.Compose([
    T.RandomCrop(32, padding=4),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize((0.4914, 0.4822, 0.4465),
                (0.2023, 0.1994, 0.2010)),
])

transform_test = T.Compose([
    T.ToTensor(),
    T.Normalize((0.4914, 0.4822, 0.4465),
                (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root="./data", train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root="./data", train=False,
                                       download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=1,
                                         shuffle=False, num_workers=2)


100%|██████████| 170M/170M [00:05<00:00, 29.6MB/s]


In [ ]:
def evaluate(model):
    model.eval()
    correct, total = 0, 0

    with torch.no_grad():
        for x, y in testloader:
            x, y = x.to(device), y.to(device)
            preds = model(x).argmax(1)
            correct += (preds == y).sum().item()
            total += y.size(0)

    return correct / total

In [ ]:
# train(model, epochs=200, patience=15)
model.load_state_dict(torch.load("best_resnet_cifar.pt", map_location="cpu"))

# model.load_state_dict(torch.load("best_resnet_cifar.pt"))
# Final test accuracy
final_acc = evaluate(model)
print(f"Final Test Accuracy: {final_acc*100:.2f}%")

Final Test Accuracy: 86.42%


In [ ]:
def evaluate_full_cifar(model, testloader, attack_fn, max_samples=None):

    device = next(model.parameters()).device

    success = 0
    distortions = []
    queries_list = []
    total = 0

    for x, y in tqdm(testloader, desc="Evaluating CIFAR"):

        x_gpu = x.to(device)
        y_gpu = y.to(device)

        # Skip wrongly classified
        with torch.no_grad():
            pred = model(x_gpu).argmax(1).item()
        if pred != y.item():
            continue

        # Prepare numpy (3,32,32)
        x_np = x.squeeze(0).cpu().numpy()
        true_label = y.item()

        # RUN ATTACK
        delta_flat, diag = attack_fn(model, x_np, true_label)
        queries = diag["queries"]

        # Build adversarial
        adv = x_np + delta_flat.reshape(3,32,32)
        adv_t = torch.from_numpy(adv).float().unsqueeze(0).to(device)

        with torch.no_grad():
            adv_pred = model(adv_t).argmax(1).item()

        total += 1
        dist = np.linalg.norm(delta_flat)

        if adv_pred != true_label:
            success += 1
            distortions.append(dist)
            queries_list.append(queries)

        if max_samples and total >= max_samples:
            break

    return {
        "total_attacked": total,
        "success_rate": success / total,
        "distortions": distortions,
        "queries": queries_list
    }


def attack_wrapper(model):
    def attack_fn(model, x0_CHW, true_label):
        # NEEDS HYPERPARAM TUNING
        delta_flat, diag = vr_zo_adamm(
            model_or_predict=model,
            x0=x0_CHW,             # shape: (3,32,32)
            true_label=true_label,
            mu=0.001,
            q=6,
            Q_ref=256,
            inner_m=10,
            epochs=3,
            alpha=0.005,
            constrained=True,
            lb=-1.0,
            ub=1.0,
            verbose=False
        )
        return delta_flat, diag
    return attack_fn

attack_fn = attack_wrapper(model)

In [ ]:
results = evaluate_full_cifar(model, testloader, attack_fn, max_samples=200)

print("Attacked samples:", results["total_attacked"])
print("Success rate:", results["success_rate"])
print("Avg distortion:", np.mean(results["distortions"]))
print("Avg queries:", np.mean(results["queries"]))

Evaluating CIFAR:   2%|▏         | 230/10000 [02:24<1:42:12,  1.59it/s]

Attacked samples: 200
Success rate: 0.905
Avg distortion: 29.059635
Avg queries: 2101.9116022099447


In [ ]:
# ZO SGD
results = evaluate_full_cifar(model, testloader, attack_fn, max_samples=30)

print("Attacked samples:", results["total_attacked"])
print("Success rate:", results["success_rate"])
print("Avg distortion:", np.mean(results["distortions"]))
print("Avg queries:", np.mean(results["queries"]))

Evaluating CIFAR:   0%|          | 30/10000 [05:02<27:54:18, 10.08s/it]

Attacked samples: 30
Success rate: 0.9333333333333333
Avg distortion: 30.522812
Avg queries: 358.89285714285717


In [ ]:
import itertools
import pandas as pd
import csv
from datetime import datetime

# ---------------------------------------
# Define search ranges
# ---------------------------------------
hyper_space = {
    "mu": [0.0005, 0.001, 0.002, 0.003, 0.004, 0.005],
    "q": [4, 6, 8, 10],
    "alpha": [0.001, 0.002, 0.005],
    "inner_m": [5, 10],
    "epochs": [2, 3]
}

# Cartesian product of all hyperparameter combos
keys = list(hyper_space.keys())
search_list = list(itertools.product(*hyper_space.values()))

# ---------------------------------------
# CSV output file
# ---------------------------------------
csv_file = "zoadamm_svrg_hyperparam_results.csv"

# Create CSV header
header = keys + [
    "success_rate",
    "avg_distortion",
    "avg_queries",
    "total_attacked",
    "timestamp"
]

# Start fresh
with open(csv_file, "w", newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)

# ---------------------------------------
# Loop over combinations
# ---------------------------------------
for values in search_list:

    params = dict(zip(keys, values))

    print("\n====================================")
    print("Running hyperparameters:", params)
    print("====================================")

    # Build attack_fn with these parameters
    def attack_wrapper_hp(model):
        def attack_fn(model, x0_CHW, true_label):
            delta, diag = vr_zo_adamm(
                model_or_predict=model,
                x0=x0_CHW,
                true_label=true_label,
                mu=params["mu"],
                q=params["q"],
                inner_m=params["inner_m"],
                epochs=params["epochs"],
                alpha=params["alpha"],
                Q_ref=256,
                constrained=True,
                lb=-1.0,
                ub=1.0,
                verbose=False
            )
            return delta, diag
        return attack_fn

    attack_fn_hp = attack_wrapper_hp(model)

    # Evaluate
    results = evaluate_full_cifar(
        model=model,
        testloader=testloader,
        attack_fn=attack_fn_hp,
        max_samples=30
    )

    success_rate   = results["success_rate"]
    avg_distortion = float(np.mean(results["distortions"])) if results["distortions"] else None
    avg_queries    = float(np.mean(results["queries"])) if results["queries"] else None
    total_attacked = results["total_attacked"]

    # Print metrics
    print("Success rate:", success_rate)
    print("Avg distortion:", avg_distortion)
    print("Avg queries:", avg_queries)
    # ---------------------------------------
    # Write row to CSV
    # ---------------------------------------
    row = list(values) + [
        success_rate,
        avg_distortion,
        avg_queries,
        total_attacked,
        datetime.now().isoformat()
    ]

    with open(csv_file, "a", newline='') as f:
        writer = csv.writer(f)
        writer.writerow(row)

    print(f"✔ Logged results to {csv_file}")



Running hyperparameters: {'mu': 0.0005, 'q': 4, 'alpha': 0.001, 'inner_m': 5, 'epochs': 2}


Evaluating CIFAR:   4%|▍         | 378/10000 [00:14<06:07, 26.20it/s]


Success rate: 0.6333333333333333
Avg distortion: 30.39799690246582
Avg queries: 1194.0
✔ Logged results to zoadamm_svrg_hyperparam_results.csv

Running hyperparameters: {'mu': 0.0005, 'q': 4, 'alpha': 0.001, 'inner_m': 5, 'epochs': 3}


Evaluating CIFAR:   4%|▍         | 378/10000 [00:18<07:50, 20.47it/s]


Success rate: 0.7
Avg distortion: 29.566909790039062
Avg queries: 1791.0
✔ Logged results to zoadamm_svrg_hyperparam_results.csv

Running hyperparameters: {'mu': 0.0005, 'q': 4, 'alpha': 0.001, 'inner_m': 10, 'epochs': 2}


...

Success rate: 0.9
Avg distortion: 31.105457305908203
Avg queries: 2028.0
✔ Logged results to zoadamm_svrg_hyperparam_results.csv

Running hyperparameters: {'mu': 0.005, 'q': 10, 'alpha': 0.005, 'inner_m': 10, 'epochs': 2}


Evaluating CIFAR:   4%|▍         | 378/10000 [00:19<08:23, 19.12it/s]


Success rate: 1.0
Avg distortion: 32.32917785644531
Avg queries: 1844.0
✔ Logged results to zoadamm_svrg_hyperparam_results.csv

Running hyperparameters: {'mu': 0.005, 'q': 10, 'alpha': 0.005, 'inner_m': 10, 'epochs': 3}


Evaluating CIFAR:   4%|▍         | 378/10000 [00:23<10:05, 15.89it/s]

Success rate: 1.0
Avg distortion: 32.33631896972656
Avg queries: 2397.0
✔ Logged results to zoadamm_svrg_hyperparam_results.csv


In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1), nn.ReLU(),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2),                # Output: (64, 14, 14)
        )
        self.fc = nn.Sequential(
            nn.Linear(64*14*14, 128), nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        z = self.conv(x)              # (B, 64, 14, 14)
        z = z.view(z.size(0), -1)     # Flatten correctly
        return self.fc(z)

In [ ]:
model = SimpleCNN().to(device)

# MNIST loading
transform = T.Compose([
    T.ToTensor(),
    T.Lambda(lambda x: x - 0.5)
])

trainset = torchvision.datasets.MNIST("./data", train=True, download=True, transform=transform)
testset  = torchvision.datasets.MNIST("./data", train=False, download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testloader  = torch.utils.data.DataLoader(testset, batch_size=256, shuffle=False)

In [ ]:
model.load_state_dict(torch.load("best_mnist_cnn.pt"))

def evaluate_full_mnist(model, testloader, attack_fn, max_samples=None):
    """
    attack_fn(model, x0 (C,H,W), true_label) → (delta_flat, diagnostics)
    Supports ANY batch size from the testloader.
    """

    device = next(model.parameters()).device

    success = 0
    distortions = []
    queries_list = []
    total = 0

    for X, Y in tqdm(testloader, desc="Evaluating MNIST"):

        # Move whole batch to device
        X = X.to(device)     # shape (B,1,28,28)
        Y = Y.to(device)     # shape (B,)

        # Clean predictions for whole batch
        with torch.no_grad():
            preds = model(X).argmax(1)        # shape (B,)

        # Loop over each element in batch
        batch_size = X.size(0)

        for i in range(batch_size):

            if preds[i].item() != Y[i].item():
                continue   # skip misclassified samples

            # Prepare CPU np array for attack
            x_i_cpu = X[i].cpu().numpy()   # (1,28,28)
            y_i_cpu = Y[i].cpu().item()

            # Run attack
            delta_flat, diag = attack_fn(model, x_i_cpu, y_i_cpu)
            queries = diag["queries"]

            # Build adversarial sample
            adv = x_i_cpu + delta_flat.reshape(1, 28, 28)
            adv_t = torch.from_numpy(adv).float().unsqueeze(0).to(device)

            # Predict adversarial label
            with torch.no_grad():
                adv_pred = model(adv_t).argmax(1).item()

            total += 1
            dist = np.linalg.norm(delta_flat)

            if adv_pred != y_i_cpu:
                success += 1
                distortions.append(dist)
                queries_list.append(queries)

            if max_samples and total >= max_samples:
                break

        if max_samples and total >= max_samples:
            break

    success_rate = success / total if total > 0 else 0

    return {
        "success_rate": success_rate,
        "distortions": distortions,
        "queries": queries_list,
        "total_attacked": total
    }


# ----------------------------------------------------------
#  Attack wrapper for the generalized VR–ZO–AdamM
# ----------------------------------------------------------

def attack_wrapper(model):
    def attack_fn(model, x0_CHW, true_label):
        # x0_CHW: numpy array (C,H,W), e.g. (1,28,28)
        delta_flat, diag = vr_zo_adamm(
            model_or_predict=model,
            x0=x0_CHW,                      # no flattening
            true_label=true_label,
            mu=0.01,
            q=64,
            Q_ref=256,
            inner_m=20,
            epochs=5,
            alpha=0.05,
            constrained=True,
            lb=-0.5,
            ub=0.5,
            verbose=False
        )
        return delta_flat, diag

    return attack_fn


attack_fn = attack_wrapper(model)

In [ ]:
import itertools
import pandas as pd
import csv
from datetime import datetime

# ---------------------------------------
# Define search ranges
# ---------------------------------------
# hyper_space = {
#     "mu": [0.0005, 0.001, 0.002, 0.003, 0.004, 0.005],
#     "q": [4, 6, 8, 10],
#     "alpha": [0.001, 0.002, 0.005],
#     "inner_m": [5, 10],
#     "epochs": [2, 3]
# }
hyper_space = {
    "mu": [0.005, 0.01, 0.02, 0.03],
    "q": [32,64],
    "alpha": [0.01, 0.02, 0.05, 0.1],
    "inner_m": [5, 10],
    "epochs": [2, 3]
}

# Cartesian product of all hyperparameter combos
keys = list(hyper_space.keys())
search_list = list(itertools.product(*hyper_space.values()))

# ---------------------------------------
# CSV output file
# ---------------------------------------
csv_file = "mnist_zoadamm_svrg_hyperparam_results.csv"

# Create CSV header
header = keys + [
    "success_rate",
    "avg_distortion",
    "avg_queries",
    "total_attacked",
    "timestamp"
]

# Start fresh
with open(csv_file, "w", newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)

# ---------------------------------------
# Loop over combinations
# ---------------------------------------
for values in search_list:

    params = dict(zip(keys, values))

    print("\n====================================")
    print("Running hyperparameters:", params)
    print("====================================")

    # Build attack_fn with these parameters
    def attack_wrapper_hp(model):
        def attack_fn(model, x0_CHW, true_label):
            delta, diag = vr_zo_adamm(
                model_or_predict=model,
                x0=x0_CHW,
                true_label=true_label,
                mu=params["mu"],
                q=params["q"],
                inner_m=params["inner_m"],
                epochs=params["epochs"],
                alpha=params["alpha"],
                Q_ref=256,
                constrained=True,
                lb=-1.0,
                ub=1.0,
                verbose=False
            )
            return delta, diag
        return attack_fn

    attack_fn_hp = attack_wrapper_hp(model)

    # Evaluate
    results = evaluate_full_mnist(
        model=model,
        testloader=testloader,
        attack_fn=attack_fn_hp,
        max_samples=None
    )

    success_rate   = results["success_rate"]
    avg_distortion = float(np.mean(results["distortions"])) if results["distortions"] else None
    avg_queries    = float(np.mean(results["queries"])) if results["queries"] else None
    total_attacked = results["total_attacked"]

    # Print metrics
    print("Success rate:", success_rate)
    print("Avg distortion:", avg_distortion)
    print("Avg queries:", avg_queries)
    # ---------------------------------------
    # Write row to CSV
    # ---------------------------------------
    row = list(values) + [
        success_rate,
        avg_distortion,
        avg_queries,
        total_attacked,
        datetime.now().isoformat()
    ]

    with open(csv_file, "a", newline='') as f:
        writer = csv.writer(f)
        writer.writerow(row)

    print(f"✔ Logged results to {csv_file}")



Running hyperparameters: {'mu': 0.005, 'q': 32, 'alpha': 0.01, 'inner_m': 5, 'epochs': 2}


Evaluating MNIST: 100%|██████████| 40/40 [10:38<00:00, 15.97s/it]


Success rate: 0.09127543824299819
Avg distortion: 1.6409168243408203
Avg queries: 1566.9558498896247
✔ Logged results to mnist_zoadamm_svrg_hyperparam_results.csv

Running hyperparameters: {'mu': 0.005, 'q': 32, 'alpha': 0.01, 'inner_m': 5, 'epochs': 3}


Evaluating MNIST: 100%|██████████| 40/40 [10:38<00:00, 15.97s/it]


Success rate: 0.09127543824299819
Avg distortion: 1.6409168243408203
Avg queries: 1566.9558498896247
✔ Logged results to mnist_zoadamm_svrg_hyperparam_results.csv

Running hyperparameters: {'mu': 0.005, 'q': 32, 'alpha': 0.01, 'inner_m': 10, 'epochs': 2}


...

Evaluating MNIST: 100%|██████████| 40/40 [09:31<00:00, 14.30s/it]


Success rate: 0.05853314527503526
Avg distortion: 1.5844889879226685
Avg queries: 1655.0086058519794
✔ Logged results to mnist_zoadamm_svrg_hyperparam_results.csv

Running hyperparameters: {'mu': 0.005, 'q': 64, 'alpha': 0.01, 'inner_m': 10, 'epochs': 3}


Evaluating MNIST: 100%|██████████| 40/40 [09:32<00:00, 14.31s/it]


Success rate: 0.05853314527503526
Avg distortion: 1.5844889879226685
Avg queries: 1655.0086058519794
✔ Logged results to mnist_zoadamm_svrg_hyperparam_results.csv

Running hyperparameters: {'mu': 0.005, 'q': 64, 'alpha': 0.02, 'inner_m': 5, 'epochs': 2}


Evaluating MNIST:  65%|██████▌   | 26/40 [07:02<03:50, 16.45s/it]